# Standing Fan EDA

### Speed of fan blades
- It was determined by Android app: Spectroid 
    - https://play.google.com/store/apps/details?id=org.intoorbit.spectrum&hl=en
- Audo settings:
    - Sampling rate: 48 kHz
    - FFT size: 8192 bins (5.9 Hz/bin)
    - Decimations: 5 (0.18 Hz/bin @ DC)
    - Window function: Hann
    - Transform interval: 50 ms
    - Exponential smooting factor: 0.5
- Fan blade speed and rotational speed (3 blades)
    - Speed 1: 57 Hz / 3 = 19 Hz
    - Speed 2: 63 Hz / 3 = 21 Hz
    - Speed 3: 68 Hz / 3 = 22.7 Hz

### Import audio

In [ ]:
import os
from scipy.io import wavfile
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks, welch

In [ ]:
path = '../../datasets/StandingFan/AUDIO/'
filenames = ('1.wav', '2.wav', '3.wav')

audio = []

for name in filenames:
    name = os.path.join(path, name)
    samplerate, data = wavfile.read(name)
    n = data.shape[0]
    duration =  n / samplerate
    channels = data.shape[1] if len(data.shape) > 1 else 1

    audio.append({
        'fs': samplerate,
        'duration': duration,
        'channels': channels,
        'n': n,
        'stream': data
    })

pd.DataFrame.from_records(audio)

In [ ]:
window_size = 2 ** 18
blades = 3
fig, ax = plt.subplots(3, 1, figsize=(9, 6))
for i, s in enumerate(audio):
    fundamental = plot_psd(ax[i], s['stream'], s['fs'], window_size)
    resolution = s['fs'] / window_size
    speed = fundamental / blades
    ax[i].set_title(f'Speed: {i+1}, Rotation: {speed:.4f} +/- {resolution:.4f} Hz')

fig.tight_layout()
plt.show()

### Plot audio waveforms

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(20, 8))
for i, s in enumerate(audio):
    t = np.linspace(0, s['duration'], s['n'])
    ax[i][0].plot(t, s['stream'])

    x = 15
    d = 0.3
    t = np.linspace(x, x + d, int(s['fs'] * d))
    y = s['stream'][x * s['fs']: int((x + d) * s['fs'])]
    ax[i][1].plot(t, y)

    for j in range(2):
        ax[i][j].set_xlabel('Time [s]')
        ax[i][j].set_ylabel('Amplitude')
        ax[i][j].set_title(f'Speed: {i+1}')

fig.tight_layout()
plt.show()

### Plot audio frequency spectrum
- Results:
    - Speed 1: 19 Hz
    - Speed 2: 21 Hz
    - Speed 3: 22.5 Hz

In [ ]:
def spectral_transform(v: pd.DataFrame, window: int, fs: int) -> tuple:
    overlap = 0.5
    step = int(window * overlap)
    return welch(
        v,
        fs=fs,
        window='hann',
        nperseg=window,
        noverlap=step,
        scaling='spectrum',
        average='mean',
        detrend='constant',
        return_onesided=True
    )

def plot_psd(ax, ts: pd.DataFrame, fs: int, window: int, threshold: int = None):
    freqs, pxx = spectral_transform(ts, window, fs)
    ax.plot(freqs, pxx, color='darkblue')
    ax.fill_between(freqs, pxx, color='lightblue', alpha=0.3)
    if threshold:
        peaks, _ = find_peaks(pxx, height=threshold)
        ax.plot(freqs[peaks], pxx[peaks], 'o', color='r')
    ax.set_xlim(0, 300)

    f = freqs[peaks][0] if len(peaks) > 0 else 0
    return f

window_size = 2 ** 18
blades = 3
fig, ax = plt.subplots(3, 1, figsize=(9, 6))
for i, s in enumerate(audio):
    fundamental = plot_psd(ax[i], s['stream'], s['fs'], window_size, 4000)
    resolution = s['fs'] / window_size
    speed = fundamental / blades
    ax[i].set_title(f'Speed: {i+1}, Rotation: {speed:.4f} +/- {resolution:.4f} Hz')

fig.tight_layout()
plt.show()

In [ ]:
# Import Steval data by metadata - placement: back/side/front, speed: 1, 2, 3

g = 9.80665
fs = 26886

def csv_import(filename: str) -> pd.DataFrame:
    axis =  ['x', 'y', 'z']
    columns = ['t'] + axis

    ts = pd.read_csv(
        filename,
        delimiter='\t',
        index_col=False,
        header=0,
        names=columns
    ) 
    
    ts[axis] = ts[axis].apply(lambda x: g * (x / 1000))
    T = 1 / fs
    ts = (
        ts
        .assign(t = lambda x: x.index * T)
    )
    ts.set_index('t', inplace=True)
    return ts

In [ ]:
def load_placement(place: str):
    path = os.path.join('../../datasets/StandingFan', place.upper())
    filenames = ('1.tsv', '2.tsv', '3.tsv')
    accel = []

    for name in filenames:
        name = os.path.join(path, name)
        ts = csv_import(name)
        n = data.shape[0]
        duration =  n / samplerate
        channels = data.shape[1] if len(data.shape) > 1 else 1

        accel.append({
            'fs': fs,
            'duration': ts.tail(1).index.to_list()[0],
            'n': len(ts),
            'stream': ts
        })
    return accel

accel = load_placement('back')   # TODO: side, front
pd.DataFrame.from_records(accel)

In [ ]:
window_size = 2 ** 18
blades = 3
fig, ax = plt.subplots(3, 1, figsize=(9, 6), sharey=True)
for i, s in enumerate(accel):
    fundamental = plot_psd(ax[i], s['stream']['x'], s['fs'], window_size, 0.12)
    resolution = s['fs'] / window_size
    speed = fundamental / blades
    ax[i].set_title(f'Speed: {i+1}, Rotation: {speed:.4f} +/- {resolution:.4f} Hz')

fig.tight_layout()
plt.show()


In [ ]:
# Show TD waveform (subplot - axis, row - placement, column - speed)

In [ ]:
# Compare spectra in differnet places at same speed 